In [ ]:
#!pip install webdriver-manager

In [1]:
import re
import pandas as pd
from pandas import DataFrame 

import openpyxl
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver 
from selenium.common.exceptions import NoSuchElementException 
from selenium.webdriver.common.keys import Keys

# 시간
from tqdm import tqdm 
import time

#이미지 다운
import urllib.request

#오류
import traceback

In [2]:
#driver = webdriver.Chrome('C:/Users/Administrator/jpnt/Final Project/chromedriver')

## Webdirver option 설정
options = webdriver.ChromeOptions()
options.add_argument('headless') #크롬 띄우기창 없애기
options.add_argument('window-size=1920x1080') # 크롬드라이버 창크기 
options.add_argument("disable-gpu") #그래픽 성능 낮춰서 크롤링 성능 쪼금 높이기 
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36") # 네트워크 설정 
options.add_argument("lang=ko_KR") # 사이트 주언어 
driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=options)

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\Song Kyung Jun\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache
C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_16532/4267179865.py:10: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=options)


In [4]:
#검색할 데이터 명 저장
df = pd.read_excel("C:/Users/Song Kyung Jun/Desktop/skj/jpnt/Final Project/search_list.xlsx")

In [5]:
data_list = []
# 포문
for idx, row in tqdm(df.iterrows()): 
    cd_idx = row['cd_idx']
    search_name = row['search_list'] 
    try:
        #영상 업로드 시간을 한정하기 위해 자동으로 click하게 만든 코드
        youtubeUrl = (f"https://www.youtube.com/results?search_query={search_name}") 
        driver.get(youtubeUrl)
        time.sleep(1.5)
        driver.find_element_by_xpath('//*[@id="container"]/ytd-toggle-button-renderer/a').click()
        time.sleep(0.6)
        driver.find_element_by_css_selector("#collapse-content ytd-search-filter-group-renderer:nth-child(1) ytd-search-filter-renderer:nth-of-type(3) a").click()
        time.sleep(0.6)
        
        no_of_pagedowns = 300 #유튜브에서 한 검색어당 500개 이상은 잘 안나옴
        elem = driver.find_element_by_tag_name("body") 
        while no_of_pagedowns: 
            elem.send_keys(Keys.PAGE_DOWN) 
            time.sleep(0.5) 
            no_of_pagedowns -= 1
        boxs = driver.find_elements_by_id("dismissible")


        for box in boxs: 
            title = box.find_element_by_css_selector('#video-title') 
            link = title.get_attribute("href") 
            title_name = box.find_element_by_css_selector('#video-title').text 
            img_ = box.find_element_by_css_selector('#img') 
            img = img_.get_attribute('src')
            while img is None: #None값을 없애기 위해 강제적으로 while문 사용
                img = img_.get_attribute('src')

            name = box.find_element_by_css_selector('#channel-info').text  
            regdate_ment = box.find_element_by_css_selector('#metadata-line > span:nth-child(2)').text 
            viewcount = box.find_element_by_css_selector('#metadata-line > span:nth-child(1)').text 
            
            
            data = {
                'search_list': search_name, 
                    "link": link,
                    "title_name": title_name,
                    "image": img, "name": name, 
                    "regdate_ment": regdate_ment, "viewcount": viewcount}
            data_list.append(data)


    except: #렉 걸려서 멈췄을 때를 대비하기 위해
        print(idx) #몇번째에서 렉걸렸고 못 받아온건지 알기 위한 코드
        continue

result_df = pd.DataFrame(data_list,columns=[
    'search_list', 'link',
    'title_name', 'image','name', 
    'regdate_ment', 'viewcount']) 

1it [03:09, 189.83s/it]

0


3it [11:04, 223.36s/it]

2


4it [14:17, 211.38s/it]

...

9it [33:58, 238.62s/it]

8


10it [37:04, 222.49s/it]

9


12it [45:50, 229.17s/it]


In [6]:
result_df.to_excel(f'C:/Users/Song Kyung Jun/Desktop/skj/jpnt/Final Project/211105_result.xlsx', index=False)

In [3]:
#읽어오기, 중복값 제거
result_df = pd.read_excel('C:/Users/Song Kyung Jun/Desktop/skj/jpnt/Final Project/211105_result.xlsx')
result_df_drop = result_df.drop_duplicates()
#index reset하고 열 드랍
rir = result_df_drop.reset_index()
rir = rir.drop(['index'], axis=1)
rir
rir.to_excel(f'C:/Users/Song Kyung Jun/Desktop/skj/jpnt/Final Project/rir_fof.xlsx', index=False)

In [4]:
#None 조회에는 is를 쓴다.
pd.value_counts(result_df['search_list'])

순대 먹방       503
분모자 먹방      494
삼겹살 먹방      492
김치찌개 먹방     488
카레 먹방       482
볶음밥 먹방      476
푸드파이터 먹방    196
먹방 ASMR     173
곱창 먹방       120
중국당면 먹방      39
라면 먹방         3
디저트 먹방        3
Name: search_list, dtype: int64

In [5]:
len(rir)

3457

In [4]:
rir

,search_list,link,title_name,image,name,regdate_ment,viewcount
0,라면 먹방,https://www.youtube.com/watch?v=KyHcJfCg7d4,1년만에 다시 찾아온 웅이와 솥뚜껑 파삼라면 8개 먹방!! (Hot spicy in...,https://i.ytimg.com/vi/KyHcJfCg7d4/hq720.jpg?s...,흥삼이네 Heungsam's Family,5일 전,조회수 66만회
1,라면 먹방,https://www.youtube.com/watch?v=_N8rp--rvtI,다이어트하느라 예민한 형 옆에서 라면 먹방하기ㅋㅋㅋ,https://i.ytimg.com/vi/_N8rp--rvtI/hq720.jpg?s...,지피티,21시간 전,조회수 15만회
2,라면 먹방,https://www.youtube.com/watch?v=jUYF9iaaow0,치즈🧀 넣어 고소한 칼국수 부대찌개 먹방:) Cheese Spicy Sausage ...,https://i.ytimg.com/vi/jUYF9iaaow0/hq720.jpg?s...,[AmiAmi]아미아미,21시간 전,조회수 8.3만회
3,라면 먹방,https://www.youtube.com/watch?v=yf_xTZoLPMo,[퀵클립] ※레전드 먹방 나왔습니다※ 하민&시현의 삽겹살+라면 먹방｜내가키운다｜JT...,https://i.ytimg.com/vi/yf_xTZoLPMo/hq720.jpg?s...,JTBC Voyage,2일 전,조회수 20만회
4,라면 먹방,https://www.youtube.com/watch?v=VQiGJsysN4o,SUB) 대파 듬뿍 들어간 대왕솥뚜껑 라면 먹방!!🍜 할머니 김치까지~ ❤️ㅣ Sp...,https://i.ytimg.com/vi/VQiGJsysN4o/hq720.jpg?s...,[Nareum_TV] 나름TV,2주 전,조회수 43만회
...,...,...,...,...,...,...,...
4686,볶음밥 먹방,https://www.youtube.com/watch?v=QJirsGfBLdc,"스팸총각김치볶음밥, 치킨너겟 먹방/Spam chonggak kimchi fried ...",https://i.ytimg.com/vi/QJirsGfBLdc/hq720.jpg?s...,윤듀 yoondew,2일 전,조회수 46회
4687,볶음밥 먹방,https://www.youtube.com/watch?v=z3Uhb1Ktl78,엽떡 볶음밥과 엽떡 먹방,https://i.ytimg.com/vi/z3Uhb1Ktl78/hq720.jpg?s...,허브민트티,2일 전,조회수 2.7천회
4688,볶음밥 먹방,https://www.youtube.com/watch?v=B4aEXuua4ng,일어나자마자 김치볶음밥 만들어서 먹기,https://i.ytimg.com/vi/B4aEXuua4ng/hqdefault.j...,엉아,3일 전,조회수 18회
4689,볶음밥 먹방,https://www.youtube.com/watch?v=IwglBIYlTLc,먹방 브이로그 VLOG | 오늘도 먹고 내일도 먹고! ..다이어트는 언제 하지..!...,https://i.ytimg.com/vi/IwglBIYlTLc/hq720.jpg?s...,수욤이식혜 suyom,2일 전,조회수 38회


In [3]:
rir = pd.read_excel('C:/Users/Song Kyung Jun/Desktop/skj/jpnt/Final Project/rir_fof.xlsx')

In [19]:
rir[200:300]

,search_list,link,title_name,image,name,regdate_ment,viewcount
200,삼겹살 먹방,https://www.youtube.com/watch?v=9C8W2TmtqZM,쭈꾸미에 삼겹살은 반칙이지 볶음밥 알지..? | 초간단 초저렴 쭈꾸미볶음 매운맛 1...,https://i.ytimg.com/vi/9C8W2TmtqZM/hq720.jpg?s...,칩푸드 자취인들을 위한 쉬운 요리,4일 전,조회수 14회
201,삼겹살 먹방,https://www.youtube.com/watch?v=asE_VYUJMi4,비빔면에 고기가 빠질 수 없지 #Shorts,https://i.ytimg.com/vi/asE_VYUJMi4/hq720.jpg?s...,오케이[ OK ],4일 전,조회수 1.1천회
202,삼겹살 먹방,https://www.youtube.com/watch?v=S27epRVQFhI,과메기철이 돌아 왔습니다/혼술/자유인/먹방,https://i.ytimg.com/vi/S27epRVQFhI/hq720.jpg?s...,마카다TV,스트리밍 시간: 5일 전,조회수 4.6천회
203,삼겹살 먹방,https://www.youtube.com/watch?v=try3ml0uKZo,"SUB) 짱맛탱 버무리떡볶이(매운맛) 분식파티 (ft.계란말이김밥,쫄면,우동,튀김)...",https://i.ytimg.com/vi/try3ml0uKZo/hq720.jpg?s...,수향TV [suhyangTV],5일 전,조회수 15만회
204,삼겹살 먹방,https://www.youtube.com/watch?v=YlQ4vXRskyU,[힐링캠핑]캠핑먹방/우중철수/가을비/솔로캠핑/짜파게티 고작 한개먹방/solocamp...,https://i.ytimg.com/vi/YlQ4vXRskyU/hqdefault.j...,소흥이들,4일 전,조회수 3.1천회
...,...,...,...,...,...,...,...
295,삼겹살 먹방,https://www.youtube.com/watch?v=oPrrDStMtEc,"먹방 브이로그🍕 주기적으로 생각나는 음식은 떡볶이, 치킨, 마라탕, 빵이야🧇 (뿌링...",https://i.ytimg.com/vi/oPrrDStMtEc/hq720.jpg?s...,나나 Nanasserie,5일 전,조회수 5.9만회
296,삼겹살 먹방,https://www.youtube.com/watch?v=3H64MfUn0kY,(ENG SUB)soir vlog | 냉동삼겹살 볶음우동 도토리묵사발 오징어무국 냉...,https://i.ytimg.com/vi/3H64MfUn0kY/hqdefault.j...,soir수아르,5일 전,조회수 446회
297,삼겹살 먹방,https://www.youtube.com/watch?v=U-duBKKGD5E,현직 셰프가 먹어보는 돼지 한마리에 딱 하나 나오는 그 부위..(부끄) 돼지 XX먹방,https://i.ytimg.com/vi/U-duBKKGD5E/hq720.jpg?s...,창수식당CHANGSU TABLE,3일 전,조회수 123회
298,삼겹살 먹방,https://www.youtube.com/watch?v=An86Ir21ysA,짬뽕에 짜장면 먹방,https://i.ytimg.com/vi/An86Ir21ysA/hq720.jpg?s...,조아로니 food at night,5일 전,조회수 94회


In [10]:

for idx, row in rir[3000:].iterrows():
    s_names = row['link']
    urls = (f"{s_names}") 
    try: 
        driver.get(urls)
        time.sleep(2)
        subscribers = driver.find_element_by_xpath('//*[@id="owner-sub-count"]').text
        alls = driver.find_element_by_class_name('style-scope ytd-video-primary-info-renderer')
        likes = alls.find_element_by_css_selector('#text').text
        sub_data = {'subscribers':subscribers, 'likes':likes}
        subs_list.append(sub_data)
    except:
        print(idx)
        continue
subs_df = pd.DataFrame(subs_list, columns=['subscribers','likes'])

3286


In [13]:
subs_df_all = pd.concat([subs_df_2, subs_df], ignore_index=True)
subs_df_all

,subscribers,likes
0,구독자 68만명,5.3천
1,구독자 65.1만명,5.1천
2,구독자 483만명,4.8만
3,구독자 149만명,4천
4,구독자 209만명,8.8천
...,...,...
3451,구독자 3명,3
3452,구독자 56명,12
3453,,2
3454,구독자 6명,1


In [8]:
subs_df_2 = subs_df

In [14]:
subs_df_all.to_excel('C:/Users/Song Kyung Jun/Desktop/skj/jpnt/Final Project/211105.xlsx', index=False)

In [23]:
subs_df = pd.read_excel('C:/Users/Song Kyung Jun/Desktop/skj/jpnt/Final Project/211105.xlsx')

In [24]:
rir = pd.read_excel('C:/Users/Song Kyung Jun/Desktop/skj/jpnt/Final Project/rir_fof.xlsx')

In [25]:
rir = rir.dropna(axis=0)
rir=rir.reset_index()
rir = rir.drop(['index'], axis=1)

In [26]:
all_df = pd.concat([rir, subs_df], axis=1)
all_df

,search_list,link,title_name,image,name,regdate_ment,viewcount,subscribers,likes
0,라면 먹방,https://www.youtube.com/watch?v=_sKPjEplDDo,을지로 가맥집 서울식품 짜파게티 라면 먹방 제육볶음 김치찌개 계란말이 스팸 호박전 ...,https://i.ytimg.com/vi/_sKPjEplDDo/hq720.jpg?s...,상해기SangHyuk,16시간 전,조회수 15만회,구독자 68만명,5.3천
1,라면 먹방,https://www.youtube.com/watch?v=KYIys9-d-Bo,100만명 라면 끓이게 만든 사람.. 라면6봉지+스팸 쌀밥은 못참습니다,https://i.ytimg.com/vi/KYIys9-d-Bo/hq720.jpg?s...,지피티,5일 전,조회수 38만회,구독자 65.1만명,5.1천
2,라면 먹방,https://www.youtube.com/watch?v=QWtvZ6NfNJ8,울릉도11)먹방 구경하러 다 모이셨어요🤣 우진이네 오징어통찜 라면 먹방,https://i.ytimg.com/vi/QWtvZ6NfNJ8/hq720.jpg?s...,tzuyang쯔양,2일 전,조회수 180만회,구독자 483만명,4.8만
3,삼겹살 먹방,https://www.youtube.com/watch?v=ZfTMGYhmzzc,삼겹살 혼밥하러 갔는데 최소주문 3인분이라고 걱정하시던 사장님이 놀라셨어요ㅋㅋㅋㅣ삼...,https://i.ytimg.com/vi/ZfTMGYhmzzc/hq720.jpg?s...,[Nareum_TV] 나름TV,1일 전,조회수 17만회,구독자 149만명,4천
4,삼겹살 먹방,https://www.youtube.com/watch?v=uwOgKbui3Mc,잘 익은 묵은지와 통삼겹 넣어 만든 밥도둑 묵은지 통삼겹찜! (Braised Sam...,https://i.ytimg.com/vi/uwOgKbui3Mc/hq720.jpg?s...,흥삼이네 Heungsam's Family,19시간 전,조회수 26만회,구독자 209만명,8.8천
...,...,...,...,...,...,...,...,...,...
3451,볶음밥 먹방,https://www.youtube.com/watch?v=6Z7nufWXK8E,딱 한 그릇 먹방 | 한우 육회 비빔밥 | only one dish mukbang ...,https://i.ytimg.com/vi/6Z7nufWXK8E/hq720.jpg?s...,딱 한 그릇 먹방,4일 전,조회수 46회,구독자 3명,3
3452,볶음밥 먹방,https://www.youtube.com/watch?v=fGdTvAyAB6Y,"ASMR뿌링클, 불닭볶음면, 편의점 매운핫도그 먹방│Boorinkle Chcken,...",https://i.ytimg.com/vi/fGdTvAyAB6Y/hqdefault.j...,윤월클먹방[Yunwarcle],3일 전,조회수 125회,구독자 56명,12
3453,볶음밥 먹방,https://www.youtube.com/watch?v=LfCcQm473wU,"[VLOG]🍳백수도 잘 챙겨 먹자! | 집밥 해먹는 일상 브이로그 | 📝피자빵, 계...",https://i.ytimg.com/vi/LfCcQm473wU/hq720.jpg?s...,도해은,1일 전,조회수 26회,NaN,2
3454,볶음밥 먹방,https://www.youtube.com/watch?v=o1M2BSXi1dQ,20대 백수 브이로그 상수역 라멘트럭 스탠스 커피 일상 먹방 stance coffee,https://i.ytimg.com/vi/o1M2BSXi1dQ/hq720.jpg?s...,에렝이,3일 전,조회수 23회,구독자 6명,1


In [27]:
all_df.to_excel('C:/Users/Song Kyung Jun/Desktop/skj/jpnt/Final Project/211106.xlsx', index=False)

In [2]:
all_df = pd.read_excel('C:/Users/Song Kyung Jun/Desktop/skj/jpnt/Final Project/211106.xlsx', keep_default_na =False)

In [29]:
all_df['subscribers'][67]

'구독자 5.97만명'

In [50]:
pd.value_counts(all_df['subscribers'] == '')

False    2652
True      804
Name: subscribers, dtype: int64

In [7]:
all_df = all_df[all_df.subscribers != '']

,search_list,link,title_name,image,name,regdate_ment,viewcount,subscribers,likes
0,라면 먹방,https://www.youtube.com/watch?v=_sKPjEplDDo,을지로 가맥집 서울식품 짜파게티 라면 먹방 제육볶음 김치찌개 계란말이 스팸 호박전 ...,https://i.ytimg.com/vi/_sKPjEplDDo/hq720.jpg?s...,상해기SangHyuk,16시간 전,조회수 15만회,구독자 68만명,5.3천
1,라면 먹방,https://www.youtube.com/watch?v=KYIys9-d-Bo,100만명 라면 끓이게 만든 사람.. 라면6봉지+스팸 쌀밥은 못참습니다,https://i.ytimg.com/vi/KYIys9-d-Bo/hq720.jpg?s...,지피티,5일 전,조회수 38만회,구독자 65.1만명,5.1천
2,라면 먹방,https://www.youtube.com/watch?v=QWtvZ6NfNJ8,울릉도11)먹방 구경하러 다 모이셨어요🤣 우진이네 오징어통찜 라면 먹방,https://i.ytimg.com/vi/QWtvZ6NfNJ8/hq720.jpg?s...,tzuyang쯔양,2일 전,조회수 180만회,구독자 483만명,4.8만
3,삼겹살 먹방,https://www.youtube.com/watch?v=ZfTMGYhmzzc,삼겹살 혼밥하러 갔는데 최소주문 3인분이라고 걱정하시던 사장님이 놀라셨어요ㅋㅋㅋㅣ삼...,https://i.ytimg.com/vi/ZfTMGYhmzzc/hq720.jpg?s...,[Nareum_TV] 나름TV,1일 전,조회수 17만회,구독자 149만명,4천
4,삼겹살 먹방,https://www.youtube.com/watch?v=uwOgKbui3Mc,잘 익은 묵은지와 통삼겹 넣어 만든 밥도둑 묵은지 통삼겹찜! (Braised Sam...,https://i.ytimg.com/vi/uwOgKbui3Mc/hq720.jpg?s...,흥삼이네 Heungsam's Family,19시간 전,조회수 26만회,구독자 209만명,8.8천
...,...,...,...,...,...,...,...,...,...
3450,볶음밥 먹방,https://www.youtube.com/watch?v=z2BHnQ4bjeU,"[먹방브이로그] 하루종일 먹는 백수의 집밥 자취 일상 1화 (김밥,육개장,비빔밥,된...",https://i.ytimg.com/vi/z2BHnQ4bjeU/hq720.jpg?s...,소뉴sonuu,2일 전,조회수 19회,구독자 1명,1
3451,볶음밥 먹방,https://www.youtube.com/watch?v=6Z7nufWXK8E,딱 한 그릇 먹방 | 한우 육회 비빔밥 | only one dish mukbang ...,https://i.ytimg.com/vi/6Z7nufWXK8E/hq720.jpg?s...,딱 한 그릇 먹방,4일 전,조회수 46회,구독자 3명,3
3452,볶음밥 먹방,https://www.youtube.com/watch?v=fGdTvAyAB6Y,"ASMR뿌링클, 불닭볶음면, 편의점 매운핫도그 먹방│Boorinkle Chcken,...",https://i.ytimg.com/vi/fGdTvAyAB6Y/hqdefault.j...,윤월클먹방[Yunwarcle],3일 전,조회수 125회,구독자 56명,12
3454,볶음밥 먹방,https://www.youtube.com/watch?v=o1M2BSXi1dQ,20대 백수 브이로그 상수역 라멘트럭 스탠스 커피 일상 먹방 stance coffee,https://i.ytimg.com/vi/o1M2BSXi1dQ/hq720.jpg?s...,에렝이,3일 전,조회수 23회,구독자 6명,1


In [8]:
all_df = all_df.reset_index()
all_df = all_df.drop(['index'], axis=1)

In [9]:
all_df

,search_list,link,title_name,image,name,regdate_ment,viewcount,subscribers,likes
0,라면 먹방,https://www.youtube.com/watch?v=_sKPjEplDDo,을지로 가맥집 서울식품 짜파게티 라면 먹방 제육볶음 김치찌개 계란말이 스팸 호박전 ...,https://i.ytimg.com/vi/_sKPjEplDDo/hq720.jpg?s...,상해기SangHyuk,16시간 전,조회수 15만회,구독자 68만명,5.3천
1,라면 먹방,https://www.youtube.com/watch?v=KYIys9-d-Bo,100만명 라면 끓이게 만든 사람.. 라면6봉지+스팸 쌀밥은 못참습니다,https://i.ytimg.com/vi/KYIys9-d-Bo/hq720.jpg?s...,지피티,5일 전,조회수 38만회,구독자 65.1만명,5.1천
2,라면 먹방,https://www.youtube.com/watch?v=QWtvZ6NfNJ8,울릉도11)먹방 구경하러 다 모이셨어요🤣 우진이네 오징어통찜 라면 먹방,https://i.ytimg.com/vi/QWtvZ6NfNJ8/hq720.jpg?s...,tzuyang쯔양,2일 전,조회수 180만회,구독자 483만명,4.8만
3,삼겹살 먹방,https://www.youtube.com/watch?v=ZfTMGYhmzzc,삼겹살 혼밥하러 갔는데 최소주문 3인분이라고 걱정하시던 사장님이 놀라셨어요ㅋㅋㅋㅣ삼...,https://i.ytimg.com/vi/ZfTMGYhmzzc/hq720.jpg?s...,[Nareum_TV] 나름TV,1일 전,조회수 17만회,구독자 149만명,4천
4,삼겹살 먹방,https://www.youtube.com/watch?v=uwOgKbui3Mc,잘 익은 묵은지와 통삼겹 넣어 만든 밥도둑 묵은지 통삼겹찜! (Braised Sam...,https://i.ytimg.com/vi/uwOgKbui3Mc/hq720.jpg?s...,흥삼이네 Heungsam's Family,19시간 전,조회수 26만회,구독자 209만명,8.8천
...,...,...,...,...,...,...,...,...,...
2647,볶음밥 먹방,https://www.youtube.com/watch?v=z2BHnQ4bjeU,"[먹방브이로그] 하루종일 먹는 백수의 집밥 자취 일상 1화 (김밥,육개장,비빔밥,된...",https://i.ytimg.com/vi/z2BHnQ4bjeU/hq720.jpg?s...,소뉴sonuu,2일 전,조회수 19회,구독자 1명,1
2648,볶음밥 먹방,https://www.youtube.com/watch?v=6Z7nufWXK8E,딱 한 그릇 먹방 | 한우 육회 비빔밥 | only one dish mukbang ...,https://i.ytimg.com/vi/6Z7nufWXK8E/hq720.jpg?s...,딱 한 그릇 먹방,4일 전,조회수 46회,구독자 3명,3
2649,볶음밥 먹방,https://www.youtube.com/watch?v=fGdTvAyAB6Y,"ASMR뿌링클, 불닭볶음면, 편의점 매운핫도그 먹방│Boorinkle Chcken,...",https://i.ytimg.com/vi/fGdTvAyAB6Y/hqdefault.j...,윤월클먹방[Yunwarcle],3일 전,조회수 125회,구독자 56명,12
2650,볶음밥 먹방,https://www.youtube.com/watch?v=o1M2BSXi1dQ,20대 백수 브이로그 상수역 라멘트럭 스탠스 커피 일상 먹방 stance coffee,https://i.ytimg.com/vi/o1M2BSXi1dQ/hq720.jpg?s...,에렝이,3일 전,조회수 23회,구독자 6명,1


In [10]:
# viewcount 전처리
for i in range(len(all_df)):
    if "없음" in all_df["viewcount"][i]:
        all_df["viewcount"][i] = '0'
    elif "만회" in all_df["viewcount"][i]:
        num = re.findall("\d+", all_df["viewcount"][i])[0]
        all_df["viewcount"][i] = float(num)*10000

    elif "천회" in all_df["viewcount"][i]:
        num = re.findall("\d+", all_df["viewcount"][i])[0]
        all_df["viewcount"][i] = float(num)*1000

    elif "회" in all_df["viewcount"][i]:
        num = re.findall("\d+", all_df["viewcount"][i])[0]
        all_df["viewcount"][i] = float(num)

In [11]:
# regdate_ment 전처리 업로드 날짜만 int로
for i in range(len(all_df)):
    if "분" in all_df["regdate_ment"][i]: # 분 추가함
        all_df["regdate_ment"][i] = 1
    elif "시간" in all_df["regdate_ment"][i]: 
        all_df["regdate_ment"][i] = 1

    elif "일" in all_df["regdate_ment"][i]:
        num = re.findall("\d+", all_df["regdate_ment"][i])[0] #re가 리스트 형식으로 문자열을 불러옴 따라서 문자열만 출력
        all_df["regdate_ment"][i] = num

    elif "주" in all_df["regdate_ment"][i]:
        num = re.findall("\d+", all_df["regdate_ment"][i])[0]
        all_df["regdate_ment"][i] = int(num)*7 # 문자열을 숫자로 바꾼 후 계산후 문자열로 변환

    elif "개월" in all_df["regdate_ment"][i]:
        num = re.findall("\d+", all_df["regdate_ment"][i])[0]
        all_df["regdate_ment"][i] = int(num)*30

In [12]:
# likes 전처리
for i in range(len(all_df)):
    if '.' in all_df['likes'][i]:
        if "만" in all_df["likes"][i]:
            t = all_df['likes'][i][:-1]
            all_df["likes"][i] = float(t)*10000

        elif "천" in all_df["likes"][i]:
            t = all_df['likes'][i][:-1]
            all_df["likes"][i] = float(t)*1000
    elif '' == all_df["likes"][i]:
        all_df["likes"][i] = 0.
    elif '좋아요' in all_df['likes'][i]:
        all_df['likes'][i] = 0
    elif "만" in all_df["likes"][i]:
        num = re.findall("\d+", all_df["likes"][i])[0]
        all_df["likes"][i] = float(num)*10000

    elif "천" in all_df["likes"][i]:
        num = re.findall("\d+", all_df["likes"][i])[0]
        all_df["likes"][i] = float(num)*1000
    else:
        int(all_df['likes'][i])

In [13]:
# subscribers 전처리
for i in range(len(all_df)):
    if '.' in all_df['subscribers'][i]:
        if ("만명" in all_df["subscribers"][i]) or ('만' in all_df["subscribers"][i]):
            t = all_df['subscribers'][i][3:-2]
            all_df["subscribers"][i] = float(t)*10000

        elif ("천명" in all_df["subscribers"][i]) or ('천' in all_df["subscribers"][i]):
            t = all_df['subscribers'][i][3:-2]
            all_df["subscribers"][i] = float(t)*1000

#    elif '' == all_df["subscribers"][i]:
#        all_df["subscribers"][i] = 0.

    elif ("만명" in all_df["subscribers"][i]) or ('만' in all_df["subscribers"][i]):
        num = re.findall("\d+", all_df["subscribers"][i])[0]
        all_df["subscribers"][i] = float(num)*10000

    elif ("천명" in all_df["subscribers"][i]) or ('천' in all_df["subscribers"][i]):
        num = re.findall("\d+", all_df["subscribers"][i])[0]
        all_df["subscribers"][i] = float(num)*1000

    else:
        num = re.findall("\d+", all_df["subscribers"][i])[0]
        all_df["subscribers"][i] = num

In [14]:
all_df.to_excel('C:/Users/Song Kyung Jun/Desktop/skj/jpnt/Final Project/211106_preall.xlsx', index=False)

In [28]:
pre_all = pd.read_excel('C:/Users/Song Kyung Jun/Desktop/skj/jpnt/Final Project/211106_preall.xlsx')

In [16]:
pre_all_filtered = pre_all[~pre_all['title_name'].str.contains('중국 먹방', case=False)]
pre_all_filtered = pre_all_filtered[~pre_all['title_name'].str.contains('tiktok', case=False)]
pre_all_filtered = pre_all_filtered[~pre_all['title_name'].str.contains('tik tok', case=False)]
pre_all_filtered

C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_23492/926955617.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  pre_all_filtered = pre_all_filtered[~pre_all['title_name'].str.contains('tik tok', case=False)]


,search_list,link,title_name,image,name,regdate_ment,viewcount,subscribers,likes
0,라면 먹방,https://www.youtube.com/watch?v=_sKPjEplDDo,을지로 가맥집 서울식품 짜파게티 라면 먹방 제육볶음 김치찌개 계란말이 스팸 호박전 ...,https://i.ytimg.com/vi/_sKPjEplDDo/hq720.jpg?s...,상해기SangHyuk,1,150000,680000.0,5300
1,라면 먹방,https://www.youtube.com/watch?v=KYIys9-d-Bo,100만명 라면 끓이게 만든 사람.. 라면6봉지+스팸 쌀밥은 못참습니다,https://i.ytimg.com/vi/KYIys9-d-Bo/hq720.jpg?s...,지피티,5,380000,651000.0,5100
2,라면 먹방,https://www.youtube.com/watch?v=QWtvZ6NfNJ8,울릉도11)먹방 구경하러 다 모이셨어요🤣 우진이네 오징어통찜 라면 먹방,https://i.ytimg.com/vi/QWtvZ6NfNJ8/hq720.jpg?s...,tzuyang쯔양,2,1800000,4830000.0,48000
3,삼겹살 먹방,https://www.youtube.com/watch?v=ZfTMGYhmzzc,삼겹살 혼밥하러 갔는데 최소주문 3인분이라고 걱정하시던 사장님이 놀라셨어요ㅋㅋㅋㅣ삼...,https://i.ytimg.com/vi/ZfTMGYhmzzc/hq720.jpg?s...,[Nareum_TV] 나름TV,1,170000,1490000.0,4000
4,삼겹살 먹방,https://www.youtube.com/watch?v=uwOgKbui3Mc,잘 익은 묵은지와 통삼겹 넣어 만든 밥도둑 묵은지 통삼겹찜! (Braised Sam...,https://i.ytimg.com/vi/uwOgKbui3Mc/hq720.jpg?s...,흥삼이네 Heungsam's Family,1,260000,2090000.0,8800
...,...,...,...,...,...,...,...,...,...
2647,볶음밥 먹방,https://www.youtube.com/watch?v=z2BHnQ4bjeU,"[먹방브이로그] 하루종일 먹는 백수의 집밥 자취 일상 1화 (김밥,육개장,비빔밥,된...",https://i.ytimg.com/vi/z2BHnQ4bjeU/hq720.jpg?s...,소뉴sonuu,2,19,1.0,1
2648,볶음밥 먹방,https://www.youtube.com/watch?v=6Z7nufWXK8E,딱 한 그릇 먹방 | 한우 육회 비빔밥 | only one dish mukbang ...,https://i.ytimg.com/vi/6Z7nufWXK8E/hq720.jpg?s...,딱 한 그릇 먹방,4,46,3.0,3
2649,볶음밥 먹방,https://www.youtube.com/watch?v=fGdTvAyAB6Y,"ASMR뿌링클, 불닭볶음면, 편의점 매운핫도그 먹방│Boorinkle Chcken,...",https://i.ytimg.com/vi/fGdTvAyAB6Y/hqdefault.j...,윤월클먹방[Yunwarcle],3,125,56.0,12
2650,볶음밥 먹방,https://www.youtube.com/watch?v=o1M2BSXi1dQ,20대 백수 브이로그 상수역 라멘트럭 스탠스 커피 일상 먹방 stance coffee,https://i.ytimg.com/vi/o1M2BSXi1dQ/hq720.jpg?s...,에렝이,3,23,6.0,1


In [17]:
pre_all_filtered = pre_all_filtered[pre_all_filtered['subscribers'] <= 2000000]
pre_all_filtered

,search_list,link,title_name,image,name,regdate_ment,viewcount,subscribers,likes
0,라면 먹방,https://www.youtube.com/watch?v=_sKPjEplDDo,을지로 가맥집 서울식품 짜파게티 라면 먹방 제육볶음 김치찌개 계란말이 스팸 호박전 ...,https://i.ytimg.com/vi/_sKPjEplDDo/hq720.jpg?s...,상해기SangHyuk,1,150000,680000.0,5300
1,라면 먹방,https://www.youtube.com/watch?v=KYIys9-d-Bo,100만명 라면 끓이게 만든 사람.. 라면6봉지+스팸 쌀밥은 못참습니다,https://i.ytimg.com/vi/KYIys9-d-Bo/hq720.jpg?s...,지피티,5,380000,651000.0,5100
3,삼겹살 먹방,https://www.youtube.com/watch?v=ZfTMGYhmzzc,삼겹살 혼밥하러 갔는데 최소주문 3인분이라고 걱정하시던 사장님이 놀라셨어요ㅋㅋㅋㅣ삼...,https://i.ytimg.com/vi/ZfTMGYhmzzc/hq720.jpg?s...,[Nareum_TV] 나름TV,1,170000,1490000.0,4000
5,삼겹살 먹방,https://www.youtube.com/watch?v=cJuVlesOwz0,우렁강된장 대패삼겹살 청양고추 상추쌈 먹방 korean food mukbang ea...,https://i.ytimg.com/vi/cJuVlesOwz0/hq720.jpg?s...,면상호[NoodleFighter],1,9000,57500.0,448
6,삼겹살 먹방,https://www.youtube.com/watch?v=nu5Cf9VBPB4,삼겹살 먹방 아니 오겹살이네 쫀득쫀득 된장찌개 먹방 맛사운드 레전드 먹방 Samgy...,https://i.ytimg.com/vi/nu5Cf9VBPB4/hq720.jpg?s...,창배먹방 chang_bae,6,90000,546000.0,2400
...,...,...,...,...,...,...,...,...,...
2647,볶음밥 먹방,https://www.youtube.com/watch?v=z2BHnQ4bjeU,"[먹방브이로그] 하루종일 먹는 백수의 집밥 자취 일상 1화 (김밥,육개장,비빔밥,된...",https://i.ytimg.com/vi/z2BHnQ4bjeU/hq720.jpg?s...,소뉴sonuu,2,19,1.0,1
2648,볶음밥 먹방,https://www.youtube.com/watch?v=6Z7nufWXK8E,딱 한 그릇 먹방 | 한우 육회 비빔밥 | only one dish mukbang ...,https://i.ytimg.com/vi/6Z7nufWXK8E/hq720.jpg?s...,딱 한 그릇 먹방,4,46,3.0,3
2649,볶음밥 먹방,https://www.youtube.com/watch?v=fGdTvAyAB6Y,"ASMR뿌링클, 불닭볶음면, 편의점 매운핫도그 먹방│Boorinkle Chcken,...",https://i.ytimg.com/vi/fGdTvAyAB6Y/hqdefault.j...,윤월클먹방[Yunwarcle],3,125,56.0,12
2650,볶음밥 먹방,https://www.youtube.com/watch?v=o1M2BSXi1dQ,20대 백수 브이로그 상수역 라멘트럭 스탠스 커피 일상 먹방 stance coffee,https://i.ytimg.com/vi/o1M2BSXi1dQ/hq720.jpg?s...,에렝이,3,23,6.0,1


In [34]:
pre_all_filtered = pre_all_filtered.drop(['index'], axis=1)
pre_all_filtered['index'] = pre_all_filtered.index
pre_all_filtered

,level_0,search_list,link,title_name,image,name,regdate_ment,viewcount,subscribers,likes,index
0,0,라면 먹방,https://www.youtube.com/watch?v=_sKPjEplDDo,을지로 가맥집 서울식품 짜파게티 라면 먹방 제육볶음 김치찌개 계란말이 스팸 호박전 ...,https://i.ytimg.com/vi/_sKPjEplDDo/hq720.jpg?s...,상해기SangHyuk,1.0,150000.0,680000.0,5300.0,0
1,1,라면 먹방,https://www.youtube.com/watch?v=KYIys9-d-Bo,100만명 라면 끓이게 만든 사람.. 라면6봉지+스팸 쌀밥은 못참습니다,https://i.ytimg.com/vi/KYIys9-d-Bo/hq720.jpg?s...,지피티,5.0,380000.0,651000.0,5100.0,1
2,2,삼겹살 먹방,https://www.youtube.com/watch?v=ZfTMGYhmzzc,삼겹살 혼밥하러 갔는데 최소주문 3인분이라고 걱정하시던 사장님이 놀라셨어요ㅋㅋㅋㅣ삼...,https://i.ytimg.com/vi/ZfTMGYhmzzc/hq720.jpg?s...,[Nareum_TV] 나름TV,1.0,170000.0,1490000.0,4000.0,2
3,3,삼겹살 먹방,https://www.youtube.com/watch?v=cJuVlesOwz0,우렁강된장 대패삼겹살 청양고추 상추쌈 먹방 korean food mukbang ea...,https://i.ytimg.com/vi/cJuVlesOwz0/hq720.jpg?s...,면상호[NoodleFighter],1.0,9000.0,57500.0,448.0,3
4,4,삼겹살 먹방,https://www.youtube.com/watch?v=nu5Cf9VBPB4,삼겹살 먹방 아니 오겹살이네 쫀득쫀득 된장찌개 먹방 맛사운드 레전드 먹방 Samgy...,https://i.ytimg.com/vi/nu5Cf9VBPB4/hq720.jpg?s...,창배먹방 chang_bae,6.0,90000.0,546000.0,2400.0,4
...,...,...,...,...,...,...,...,...,...,...,...
2474,2478,볶음밥 먹방,https://www.youtube.com/watch?v=z2BHnQ4bjeU,"[먹방브이로그] 하루종일 먹는 백수의 집밥 자취 일상 1화 (김밥,육개장,비빔밥,된...",https://i.ytimg.com/vi/z2BHnQ4bjeU/hq720.jpg?s...,소뉴sonuu,2.0,19.0,1.0,1.0,2474
2475,2479,볶음밥 먹방,https://www.youtube.com/watch?v=6Z7nufWXK8E,딱 한 그릇 먹방 | 한우 육회 비빔밥 | only one dish mukbang ...,https://i.ytimg.com/vi/6Z7nufWXK8E/hq720.jpg?s...,딱 한 그릇 먹방,4.0,46.0,3.0,3.0,2475
2476,2480,볶음밥 먹방,https://www.youtube.com/watch?v=fGdTvAyAB6Y,"ASMR뿌링클, 불닭볶음면, 편의점 매운핫도그 먹방│Boorinkle Chcken,...",https://i.ytimg.com/vi/fGdTvAyAB6Y/hqdefault.j...,윤월클먹방[Yunwarcle],3.0,125.0,56.0,12.0,2476
2477,2481,볶음밥 먹방,https://www.youtube.com/watch?v=o1M2BSXi1dQ,20대 백수 브이로그 상수역 라멘트럭 스탠스 커피 일상 먹방 stance coffee,https://i.ytimg.com/vi/o1M2BSXi1dQ/hq720.jpg?s...,에렝이,3.0,23.0,6.0,1.0,2477


In [35]:
pre_all_filtered['index'] = pre_all_filtered['index'] + 7503

In [36]:
pre_all_filtered

,level_0,search_list,link,title_name,image,name,regdate_ment,viewcount,subscribers,likes,index
0,0,라면 먹방,https://www.youtube.com/watch?v=_sKPjEplDDo,을지로 가맥집 서울식품 짜파게티 라면 먹방 제육볶음 김치찌개 계란말이 스팸 호박전 ...,https://i.ytimg.com/vi/_sKPjEplDDo/hq720.jpg?s...,상해기SangHyuk,1.0,150000.0,680000.0,5300.0,7503
1,1,라면 먹방,https://www.youtube.com/watch?v=KYIys9-d-Bo,100만명 라면 끓이게 만든 사람.. 라면6봉지+스팸 쌀밥은 못참습니다,https://i.ytimg.com/vi/KYIys9-d-Bo/hq720.jpg?s...,지피티,5.0,380000.0,651000.0,5100.0,7504
2,2,삼겹살 먹방,https://www.youtube.com/watch?v=ZfTMGYhmzzc,삼겹살 혼밥하러 갔는데 최소주문 3인분이라고 걱정하시던 사장님이 놀라셨어요ㅋㅋㅋㅣ삼...,https://i.ytimg.com/vi/ZfTMGYhmzzc/hq720.jpg?s...,[Nareum_TV] 나름TV,1.0,170000.0,1490000.0,4000.0,7505
3,3,삼겹살 먹방,https://www.youtube.com/watch?v=cJuVlesOwz0,우렁강된장 대패삼겹살 청양고추 상추쌈 먹방 korean food mukbang ea...,https://i.ytimg.com/vi/cJuVlesOwz0/hq720.jpg?s...,면상호[NoodleFighter],1.0,9000.0,57500.0,448.0,7506
4,4,삼겹살 먹방,https://www.youtube.com/watch?v=nu5Cf9VBPB4,삼겹살 먹방 아니 오겹살이네 쫀득쫀득 된장찌개 먹방 맛사운드 레전드 먹방 Samgy...,https://i.ytimg.com/vi/nu5Cf9VBPB4/hq720.jpg?s...,창배먹방 chang_bae,6.0,90000.0,546000.0,2400.0,7507
...,...,...,...,...,...,...,...,...,...,...,...
2474,2478,볶음밥 먹방,https://www.youtube.com/watch?v=z2BHnQ4bjeU,"[먹방브이로그] 하루종일 먹는 백수의 집밥 자취 일상 1화 (김밥,육개장,비빔밥,된...",https://i.ytimg.com/vi/z2BHnQ4bjeU/hq720.jpg?s...,소뉴sonuu,2.0,19.0,1.0,1.0,9977
2475,2479,볶음밥 먹방,https://www.youtube.com/watch?v=6Z7nufWXK8E,딱 한 그릇 먹방 | 한우 육회 비빔밥 | only one dish mukbang ...,https://i.ytimg.com/vi/6Z7nufWXK8E/hq720.jpg?s...,딱 한 그릇 먹방,4.0,46.0,3.0,3.0,9978
2476,2480,볶음밥 먹방,https://www.youtube.com/watch?v=fGdTvAyAB6Y,"ASMR뿌링클, 불닭볶음면, 편의점 매운핫도그 먹방│Boorinkle Chcken,...",https://i.ytimg.com/vi/fGdTvAyAB6Y/hqdefault.j...,윤월클먹방[Yunwarcle],3.0,125.0,56.0,12.0,9979
2477,2481,볶음밥 먹방,https://www.youtube.com/watch?v=o1M2BSXi1dQ,20대 백수 브이로그 상수역 라멘트럭 스탠스 커피 일상 먹방 stance coffee,https://i.ytimg.com/vi/o1M2BSXi1dQ/hq720.jpg?s...,에렝이,3.0,23.0,6.0,1.0,9980


In [37]:
pre_all_filtered.to_excel('C:/Users/Song Kyung Jun/Desktop/skj/jpnt/Final Project/211106_preall_fil.xlsx', index=False)

In [38]:
#이미지 저장 코드
#pass / continue : continue 는 바로 다음 iteerows로
#pass 는 다음 명령어로
for idx, row in pre_all_filtered.iterrows():
    try:
        img = row['image']
        urllib.request.urlretrieve(img, f"{row['search_list'],row['index']}.jpg")
    except:
        print(idx)

In [30]:
pre_all_filtered=pd.read_excel('C:/Users/Song Kyung Jun/Desktop/skj/jpnt/Final Project/211106_preall_fil.xlsx')
pre_all_filtered = pre_all_filtered.dropna(axis=0)
pre_all_filtered = pre_all_filtered.reset_index()

In [63]:
pre_all_filtered=pd.read_excel('C:/Users/Song Kyung Jun/Desktop/skj/jpnt/Final Project/pre_all_filtered.xlsx')

In [32]:
pre_all_filtered

,level_0,search_list,link,title_name,image,name,regdate_ment,viewcount,subscribers,likes,index
0,0,라면 먹방,https://www.youtube.com/watch?v=_sKPjEplDDo,을지로 가맥집 서울식품 짜파게티 라면 먹방 제육볶음 김치찌개 계란말이 스팸 호박전 ...,https://i.ytimg.com/vi/_sKPjEplDDo/hq720.jpg?s...,상해기SangHyuk,1.0,150000.0,680000.0,5300.0,7503.0
1,1,라면 먹방,https://www.youtube.com/watch?v=KYIys9-d-Bo,100만명 라면 끓이게 만든 사람.. 라면6봉지+스팸 쌀밥은 못참습니다,https://i.ytimg.com/vi/KYIys9-d-Bo/hq720.jpg?s...,지피티,5.0,380000.0,651000.0,5100.0,7504.0
2,2,삼겹살 먹방,https://www.youtube.com/watch?v=ZfTMGYhmzzc,삼겹살 혼밥하러 갔는데 최소주문 3인분이라고 걱정하시던 사장님이 놀라셨어요ㅋㅋㅋㅣ삼...,https://i.ytimg.com/vi/ZfTMGYhmzzc/hq720.jpg?s...,[Nareum_TV] 나름TV,1.0,170000.0,1490000.0,4000.0,7505.0
3,3,삼겹살 먹방,https://www.youtube.com/watch?v=cJuVlesOwz0,우렁강된장 대패삼겹살 청양고추 상추쌈 먹방 korean food mukbang ea...,https://i.ytimg.com/vi/cJuVlesOwz0/hq720.jpg?s...,면상호[NoodleFighter],1.0,9000.0,57500.0,448.0,7506.0
4,4,삼겹살 먹방,https://www.youtube.com/watch?v=nu5Cf9VBPB4,삼겹살 먹방 아니 오겹살이네 쫀득쫀득 된장찌개 먹방 맛사운드 레전드 먹방 Samgy...,https://i.ytimg.com/vi/nu5Cf9VBPB4/hq720.jpg?s...,창배먹방 chang_bae,6.0,90000.0,546000.0,2400.0,7507.0
...,...,...,...,...,...,...,...,...,...,...,...
2474,2478,볶음밥 먹방,https://www.youtube.com/watch?v=z2BHnQ4bjeU,"[먹방브이로그] 하루종일 먹는 백수의 집밥 자취 일상 1화 (김밥,육개장,비빔밥,된...",https://i.ytimg.com/vi/z2BHnQ4bjeU/hq720.jpg?s...,소뉴sonuu,2.0,19.0,1.0,1.0,9981.0
2475,2479,볶음밥 먹방,https://www.youtube.com/watch?v=6Z7nufWXK8E,딱 한 그릇 먹방 | 한우 육회 비빔밥 | only one dish mukbang ...,https://i.ytimg.com/vi/6Z7nufWXK8E/hq720.jpg?s...,딱 한 그릇 먹방,4.0,46.0,3.0,3.0,9982.0
2476,2480,볶음밥 먹방,https://www.youtube.com/watch?v=fGdTvAyAB6Y,"ASMR뿌링클, 불닭볶음면, 편의점 매운핫도그 먹방│Boorinkle Chcken,...",https://i.ytimg.com/vi/fGdTvAyAB6Y/hqdefault.j...,윤월클먹방[Yunwarcle],3.0,125.0,56.0,12.0,9983.0
2477,2481,볶음밥 먹방,https://www.youtube.com/watch?v=o1M2BSXi1dQ,20대 백수 브이로그 상수역 라멘트럭 스탠스 커피 일상 먹방 stance coffee,https://i.ytimg.com/vi/o1M2BSXi1dQ/hq720.jpg?s...,에렝이,3.0,23.0,6.0,1.0,9984.0
